In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [5]:
# Import data
dataframe = pd.read_csv('datapath\\17age1.csv')
df = dataframe[['Reading']]
dataset = df.values
dataset = dataset.astype('float32')

In [6]:
# Prepare time-series data
# Creat data samples 
def create_dataset(dataset, look_back):
	dataX, dataY = [], []
	for i in range(len(dataset)-(look_back+1)):
		a = dataset[i:(i+look_back), 0]
		b = dataset[(i+look_back):(i+look_back)+2, 0]
		dataX.append(a)
		dataY.append(b)
	return np.array(dataX), np.array(dataY)
# reshape into X=t and Y=t+1
look_back = 3
X, Y = create_dataset(dataset, look_back)

In [7]:
trainX = X[:len(dataset)-3-look_back]
trainY = Y[:len(dataset)-3-look_back]
testX = X[len(X)-1]
testY = Y[len(Y)-1]

In [8]:
TrainX = trainX.reshape((trainX.shape[0], trainX.shape[1], 1))
TrainY = trainY.reshape((trainY.shape[0], trainY.shape[1], 1))
TestX  = testX.reshape(1,3,1)

In [9]:
testY

array([81. , 78.5], dtype=float32)

In [10]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(3, 1)))
model.add(RepeatVector(2))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 repeat_vector (RepeatVector  (None, 2, 100)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 2, 100)            80400     
                                                                 
 time_distributed (TimeDistr  (None, 2, 1)             101       
 ibuted)                                                         
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# fit model
model.fit(TrainX, TrainY, epochs=500, verbose=0)

In [12]:
yhat = model.predict(TestX, verbose=0)
print(yhat)

[[[81.95653]
  [78.91835]]]
